# Notebook for applying climate threat to GDEs

Use 2 datasets to determine whether GDEs are threatened by climate:

- local recharge areas
- modeled drought index



## Set up environments in ArcGIS Pro

In [ ]:
# Import ArcGIS modules and check out spatial analyst extension
import arcpy
import os
import pandas as pd
import numpy as np
from arcpy import env
from arcpy.sa import *
arcpy.CheckOutExtension("spatial")

In [ ]:
# Path to temporary geodatabase
path =  r"path_to_folder\gdbname.gdb"

# Environment settings
env.workspace = path
env.overwriteOutput = True
env.outputCoordinateSystem = arcpy.SpatialReference(26911) # Spatial reference NAD 1983 UTM Zone 11N. The code is '26911'

In [ ]:
# GDE layers
springs = r'path_to_folder\NV_iGDE_assess.gdb\Springs'
wet = r'path_to_folder\NV_iGDE_assess.gdb\Wetlands'
phr = r'path_to_folder\NV_iGDE_assess.gdb\Phreatophytes_Explode'
lp = r'path_to_folder\NV_iGDE_assess.gdb\Lakes_Playas'
rs = r'path_to_folder\NV_iGDE_assess.gdb\Rivers_Streams'


## Identify GDEs in local recharge areas

More likely to be influenced by changes in climate than GDEs in regional recharge areas. Recharge areas digitized from Mifflin 1988, Plate 3. 

Attribute: __Recharge = 0 (no threat) or 1 (threat)__

In [ ]:
# Load recharge areas
rch = r'path_to_folder\GDE_Recharge06172020.shp'
print(arcpy.GetCount_management(rch))

In [ ]:
# For each GDE type - attribute whether it intersects with a recharge area
# Springs
arcpy.AddField_management(springs, 'Recharge', 'LONG')
arcpy.CalculateField_management(springs, 'Recharge', 0)
[f.name for f in arcpy.ListFields(springs)]
spr_select = arcpy.SelectLayerByLocation_management(springs, 'INTERSECT', rch)
arcpy.CalculateField_management(spr_select, 'Recharge', 1)

# Wetlands
arcpy.AddField_management(wet, 'Recharge', 'LONG')
arcpy.CalculateField_management(wet, 'Recharge', 0)
[f.name for f in arcpy.ListFields(wet)]
wet_select = arcpy.SelectLayerByLocation_management(wet, 'INTERSECT', rch)
arcpy.CalculateField_management(wet_select, 'Recharge', 1)

# Phreatophytes
arcpy.AddField_management(phr2, 'Recharge', 'LONG')
arcpy.CalculateField_management(phr2, 'Recharge', 0)
[f.name for f in arcpy.ListFields(phr2)]
phr_select = arcpy.SelectLayerByLocation_management(phr2, 'INTERSECT', rch)
arcpy.CalculateField_management(phr_select, 'Recharge', 1)

# Lakes/playas
arcpy.AddField_management(lp, 'Recharge', 'LONG')
arcpy.CalculateField_management(lp, 'Recharge', 0)
[f.name for f in arcpy.ListFields(lp)]
lp_select = arcpy.SelectLayerByLocation_management(lp, 'INTERSECT', rch)
arcpy.CalculateField_management(lp_select, 'Recharge', 1)

# Rivers/streams
arcpy.AddField_management(rs, 'Recharge', 'LONG')
arcpy.CalculateField_management(rs, 'Recharge', 0)
[f.name for f in arcpy.ListFields(rs)]
rs_select = arcpy.SelectLayerByLocation_management(rs, 'INTERSECT', rch)
arcpy.CalculateField_management(rs_select, 'Recharge', 1)


## Basin-wide threats from modeled climate

Attribute all GDEs in basin with that basin's relative climate threat value; calculated across hydrographic basins.

Attribute: __ClmThrR = ranges 0 to 1 (less to most droughty/threatened)__

In [ ]:
# Read in basins with climate threat values (essentially "droughtiness")
clim = r'path_to_folder\BasinClimateThr.shp'
print([f.name for f in arcpy.ListFields(clim)])

In [ ]:
# Springs
spr_clim = arcpy.SpatialJoin_analysis(springs, clim, 'temp_springs_join')
[f.name for f in arcpy.ListFields(spr_clim)]
arcpy.GetCount_management(spr_clim)
arcpy.GetCount_management(springs)

arcpy.JoinField_management(springs, 'OBJECTID', spr_clim, 'OBJECTID', ['ClmThrR'])
[f.name for f in arcpy.ListFields(springs)]

# Wetlands
wet_clim = arcpy.SpatialJoin_analysis(wet, clim, 'temp_wet_join')
[f.name for f in arcpy.ListFields(wet_clim)]
arcpy.GetCount_management(wet_clim)
arcpy.GetCount_management(wet)

arcpy.JoinField_management(wet, 'OBJECTID', wet_clim, 'OBJECTID', ['ClmThrR'])
[f.name for f in arcpy.ListFields(wet)]

# Phreatophytes
phr_clim = arcpy.SpatialJoin_analysis(phr, clim, 'temp_phr_join')
[f.name for f in arcpy.ListFields(phr_clim)]
arcpy.GetCount_management(phr_clim)
arcpy.GetCount_management(phr)

arcpy.JoinField_management(phr, 'OBJECTID', phr_clim, 'OBJECTID', ['ClmThrR'])
[f.name for f in arcpy.ListFields(phr)]

# rivers/streams
rs_clim = arcpy.SpatialJoin_analysis(rs, clim, 'temp_rs_join')
[f.name for f in arcpy.ListFields(rs_clim)]
arcpy.GetCount_management(rs_clim)
arcpy.GetCount_management(rs)

arcpy.JoinField_management(rs, 'OBJECTID', rs_clim, 'OBJECTID', ['ClmThrR'])
[f.name for f in arcpy.ListFields(rs)]

# Lakes/playas
lp_clim = arcpy.SpatialJoin_analysis(lp, clim, 'temp_lp_join')
[f.name for f in arcpy.ListFields(lp_clim)]
arcpy.GetCount_management(lp_clim)
arcpy.GetCount_management(lp)

arcpy.JoinField_management(lp, 'OBJECTID', lp_clim, 'OBJECTID', ['ClmThrR'])
[f.name for f in arcpy.ListFields(lp)]